In [1]:
import numpy as np

In [2]:
# Reading and storing data from the text files

Timesheet = "timesheet.txt"
Emp_Hours = {}
with open(Timesheet, 'r') as f:
    for line in f:
        data = line.strip().split(',')
        Id = data[0]
        Hours = int(data[1])
        if Id in Emp_Hours.keys():
            Emp_Hours[Id] += Hours            
        else:
            Emp_Hours[Id]=Hours

In [3]:
EV_Sheet = "evaluation.txt"        
EVS = {}
Pos_words = ['excellent', 'good', 'dependable', 'prompt']
Neg_words = ['poor', 'error', 'unreliable', 'late']
with open(EV_Sheet, "r") as f:
    for line in f:
        data = line.strip().split("#")
        Id = data[0]
        Remark = data[1]
        Pos = 0
        for i in Pos_words:
            Pos += Remark.count(i)
        Neg = 0
        for i in Neg_words:
            Neg += Remark.count(i)  
        if Neg == 0:
            EVS[Id] = 10.0
        else:
            EVS[Id] = round(Pos/Neg, 2)

In [4]:
Sales = "sales.txt"
Dir_sales = {}
with open (Sales, "r") as f:
    for line in f:
        data = line.strip().split(',')
        Id, sale = data
        Dir_sales[Id] = int(sale)

In [7]:
Emp_data = "emp_beg_yr.txt"
Directors = {}
Consultants = {}
with open(Emp_data, "r") as f:
    f.readline()
    for line in f:
        data = line.strip().split(',')
        Id,LN,FN,JC,BP = data
        if JC == "C":
            if Id in Emp_Hours.keys():
                PUR = round((Emp_Hours[Id]/2250 * 100),1)
            else:
                PUR = 0
            if Id in EVS.keys():
                ES = EVS[Id]
            else:
                ES = 1
            BP = int(BP)
            Consultants[Id] = [Id,LN,FN,BP,PUR,ES,'C']
            
        else:
            if Id in Emp_Hours.keys():
                PUR = round((Emp_Hours[Id]/2250 * 100),1)
            else:
                PUR = 0
            if Id in Dir_sales.keys():
                AS = int(Dir_sales[Id])
            else:
                AS = 0
            Directors[Id] = [Id,LN,FN,BP,PUR,AS,"D"]

In [8]:
#Functions 
          
def Per_65(Dir, Con):
    E_PUR = []
    for i in Dir.keys():
        E_PUR.append(Dir[i][4])
    for i in Con.keys():
        E_PUR.append(Con[i][4])
    return (np.percentile(E_PUR, 65)) 

In [9]:
def Total_bonus(Dir, Con, BR):
    Total = 0
    BR = BR/100
    PUR_65 = Per_65(Dir, Con)
    for i in Dir.keys():
        if Dir[i][4] >= PUR_65:
            if Dir[i][5] * BR <= 150000:
                Total += Dir[i][5] * BR
            else:
                Total += 150000
    for i in Con.keys():
        if Con[i][4] >= PUR_65 and Con[i][5] >= 3.5:
            if Con[i][3] * BR <= 50000:
                Total += Con[i][3] * BR
            else:
                Total += 50000
    return round(Total, 2)

In [10]:
def Bonus(Dir, Con, Id, BR):
    B =0
    BR = BR/100
    PUR_65 = Per_65(Dir, Con)
    if Id in Dir.keys():
        if Dir[Id][4] >= PUR_65:
            if Dir[Id][5] * BR <= 150000:
                B = round(Dir[Id][5] * BR, 2)
            else:
                B = 150000.00
    if Id in Con.keys():
        if Con[Id][4] >= PUR_65 and Con[Id][5] >= 3.5:
            if Con[Id][3] * BR <= 50000:
                B = round(Con[Id][3] * BR ,2)
            else:
                B = 50000.00
    return B

In [11]:
def Bonus_Arr(Dir, Con, BR):
    B_Arr = []
    BR = BR/100
    PUR_65 = Per_65(Dir, Con)
    for Id in Dir.keys():
        if Dir[Id][4] >= PUR_65:
            if Dir[Id][5] * BR <= 150000:
                B_Arr.append(round(Dir[Id][5] * BR, 2))
            else:
                B_Arr.append(150000.00)
    for Id in Con.keys():
        if Con[Id][4] >= PUR_65 and Con[Id][5] >= 3.5:
            if Con[Id][3] * BR <= 50000:
                B_Arr.append(round(Con[Id][3] * BR ,2))
            else:
                B_Arr.append( 50000.00)
    return B_Arr

In [12]:
def Emp_info(Dir, Con, BR, ID):
    if ID in Dir.keys():
        print("ID:", ID)
        print("Director:" ,Dir[ID][2], Dir[ID][1])
        print("Utilization:", Dir[ID][4])
        print("New sales: $"+ str(Dir[ID][5]) )
        print("Base pay: $" + str(Dir[ID][3]))
        print("Bonus: $" + str(Bonus(Directors, Consultants,ID, BR)))
    elif ID in Con.keys():
        print("ID:", ID)
        print("Consultant:" ,Con[ID][2], Con[ID][1])
        print("Utilization:", Con[ID][4])
        print("Evaluation Score:",Con[ID][5]) 
        print("Base pay: $" + str(Con[ID][3]))
        print("Bonus: $" + str(Bonus(Directors, Consultants,ID, BR)))

In [15]:
def stat(data):
    num_points = len(data)
    min_value = np.min(data)
    max_value = np.max(data)
    median_value = np.median(data)
    mean_value = round(np.mean(data),2)
    std_dev = round(np.std(data),2)
    
    print("Number of data points:", num_points)
    print("Minimum:", min_value)
    print("Maximum:", max_value)
    print("Median:", median_value)
    print("Mean:", mean_value)
    print("Standard deviation:", std_dev)

In [17]:
while True:
    BR = float(input("Enter bonus rate: ")) 
    print("Total Bonus value for this rate is: ",Total_bonus(Directors, Consultants, BR))
    if (input("Do you want to finalize this rate? (yes/no)").lower() == 'yes' ):
        break

valid_Id = [*Directors.keys()] + [*Consultants.keys()]
valid_id_int = [int(x) for x in valid_Id]
valid_id_int.sort()
file = open("emp_end_yr.txt", "w")
file.write("ID,LastName,FirstName,JobCode,BasePay,Utilization,Evaluation/Sales,Bonus\n")
for i in valid_id_int:
    Id = str(i)
    if Id in Directors.keys():
        emp = Directors[Id]
    else:
        emp = Consultants[Id]
    text = emp[0]+','+emp[1]+','+emp[2]+','+emp[6]+','+str(emp[3])+','+str(emp[4])+','+str(emp[5])+"\n"
    file.write(text)
file.close()

Enter bonus rate:  12


Total Bonus value for this rate is:  2051513.28


Do you want to finalize this rate? (yes/no) yes


In [18]:
# 2 
    
while True:
    if (input("\nDo you want to look up for employee info? (yes/no)").lower() == 'yes'):
        ID = input("Enter employee ID: ")
        Emp_info(Directors, Consultants, BR, ID)
    else:
        break


Do you want to look up for employee info? (yes/no) yes
Enter employee ID:  12

Do you want to look up for employee info? (yes/no) yes
Enter employee ID:  1

Do you want to look up for employee info? (yes/no) no


In [19]:
# 3
    
PUR = []
for i in Directors.keys():
    PUR.append(Directors[i][4])
for i in Consultants.keys():
    PUR.append(Consultants[i][4])
    
print("\nFor Utilization rate:")
stat(PUR)
print("\nFor Evaluation score:")
stat([*EVS.values()])
print("\nFor Sales:")
stat([*Dir_sales.values()])
print("\nFor Bonus:")
stat(Bonus_Arr(Directors, Consultants, BR))   


For Utilization rate:
Number of data points: 172
Minimum: 0.0
Maximum: 120.1
Median: 91.1
Mean: 87.6
Standard deviation: 19.79

For Evaluation score:
Number of data points: 126
Minimum: 0.0
Maximum: 10.0
Median: 4.5
Mean: 5.43
Standard deviation: 4.3

For Sales:
Number of data points: 36
Minimum: 82000
Maximum: 1973000
Median: 1188500.0
Mean: 1117750.0
Standard deviation: 601233.93

For Bonus:
Number of data points: 30
Minimum: 7682.16
Maximum: 150000.0
Median: 20756.46
Mean: 68383.78
Standard deviation: 62928.63


In [20]:
# 4

Max_PUR = np.max(PUR)
Max_Sales = np.max([*Dir_sales.values()])

for i in Directors.keys():
    if Directors[i][4] == Max_PUR:
        print("\nDirector with highest utilization rate: ")
        Emp_info(Directors, Consultants, BR, i)

for i in Consultants.keys():
    if Consultants[i][4] == Max_PUR:
        print("\nConsultant with highest utilization rate: ")
        Emp_info(Directors, Consultants, BR, i)
            
for i in Directors.keys():
    if Directors[i][5] == Max_Sales:
        print("\nDirector with highest sales: ")
        Emp_info(Directors, Consultants, BR, i)    
    


Consultant with highest utilization rate: 
ID: 273
Consultant: Sandra Thompson
Utilization: 120.1
Evaluation Score: 10.0
Base pay: $67689
Bonus: $8122.68

Director with highest sales: 
ID: 104
Director: Thomas Hardy
Utilization: 99.4
New sales: $1973000
Base pay: $337573
Bonus: $150000.0


In [21]:
# 5

Threshold_UR = np.mean(PUR) - np.std(PUR)
Poor_Consultants = []

for i in Consultants.keys():
   if Consultants[i][5] < 1 and Threshold_UR > Consultants[i][4]:
       Poor_Consultants.append(i)
print("\nIDs of consultants with poor performance: ")        
print(Poor_Consultants)


IDs of consultants with poor performance: 
['216']


In [24]:
# 6 

error_log = open('error_log.txt', 'w')


f = open(Timesheet, 'r') 
error_log.write("Error log of Timesheet file:\n")
for line in f:
    data = line.strip().split(',')
    if data[0] not in valid_Id:
        error_log.write(data[0]+'\n')
f.close()
 
    
f = open(EV_Sheet, 'r') 
error_log.write("\nError log of Evaluation file:\n")

for line in f:
    data = line.strip().split('#')
    if data[0] not in valid_Id:
        error_log.write(data[0]+"\n")
f.close()  
  
f = open(Sales, 'r')
error_log.write("\nError log of Sales file:\n")
 
for line in f:
    data = line.strip().split(',')
    if data[0] not in valid_Id:
        error_log.write(data[0]+'\n')
f.close()   
error_log.close()
